In [ ]:
%%capture --no-stderr
%pip install -U  langchain_community langchain-nvidia-ai-endpoints

In [ ]:
import getpass
import os


def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")

_set_env("NVIDIA_API_KEY")

In [ ]:
from langchain_nvidia_ai_endpoints import ChatNVIDIA
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

llm = ChatNVIDIA(model="meta/llama-3.1-70b-instruct")

## 1. Basic Few-Shot

In [ ]:
def few_shot_sentiment_classification(input_text):
    few_shot_prompt = PromptTemplate(
        input_variables=["input_text"],
        template="""
        Classify the sentiment as Positive, Negative, or Neutral.

        Examples:
        Text: I love this product! It's amazing.
        Sentiment: Positive

        Text: This movie was terrible. I hated it.
        Sentiment: Negative

        Text: The weather today is okay.
        Sentiment: Neutral

        Now, classify the following:
        Text: {input_text}
        Sentiment:
        """
    )
    chain = few_shot_prompt | llm
    result = chain.invoke(input_text).content

    # Clean up the result
    result = result.strip()
    # Extract only the sentiment label
    if ':' in result:
        result = result.split(':')[1].strip()

    return result  # This will now return just "Positive", "Negative", or "Neutral"

test_text = "I can't believe how great this new restaurant is!"
result = few_shot_sentiment_classification(test_text)
print(f"Input: {test_text}")
print(f"Predicted Sentiment: {result}")

Input: I can't believe how great this new restaurant is!
Predicted Sentiment: Positive


## 2. Advanced Few-shot

In [ ]:
def multi_task_few_shot(input_text, task):
    few_shot_prompt = PromptTemplate(
        input_variables=["input_text", "task"],
        template="""
        Perform the specified task on the given text.

        Examples:
        Text: I love this product! It's amazing.
        Task: sentiment
        Result: Positive

        Text: Bonjour, comment allez-vous?
        Task: language
        Result: French

        Now, perform the following task:
        Text: {input_text}
        Task: {task}
        Result:
        """
    )

    chain = few_shot_prompt | llm
    return chain.invoke({"input_text": input_text, "task": task}).content

print(multi_task_few_shot("I can't believe how great this is!", "sentiment"))
print(multi_task_few_shot("Guten Tag, wie geht es Ihnen?", "language"))

Positive
German


## 3. In-Context Learning

In [ ]:
def in_context_learning(task_description, examples, input_text):
    example_text = "".join([f"Input: {e['input']}\nOutput: {e['output']}\n\n" for e in examples])

    in_context_prompt = PromptTemplate(
        input_variables=["task_description", "examples", "input_text"],
        template="""
        Task: {task_description}

        Examples:
        {examples}

        Now, perform the task on the following input:
        Input: {input_text}
        Output:
        """
    )

    chain = in_context_prompt | llm
    return chain.invoke({"task_description": task_description, "examples": example_text, "input_text": input_text}).content

task_desc = "Convert the given text to pig latin."
examples = [
    {"input": "hello", "output": "ellohay"},
    {"input": "apple", "output": "appleay"}
]
test_input = "python"

result = in_context_learning(task_desc, examples, test_input)
print(f"Input: {test_input}")
print(f"Output: {result}")

Input: python
Output: ythonpay


In [ ]:
def evaluate_model(model_func, test_cases):
    '''
    Evaluate the model on a set of test cases.

    Args:
    model_func: The function that makes predictions.
    test_cases: A list of dictionaries, where each dictionary contains an "input" text and a "label" for the input.

    Returns:
    The accuracy of the model on the test cases.
    '''
    correct = 0
    total = len(test_cases)

    for case in test_cases:
        input_text = case['input']
        true_label = case['label']
        prediction = model_func(input_text).strip()

        is_correct = prediction.lower() == true_label.lower()
        correct += int(is_correct)

        print(f"Input: {input_text}")
        print(f"Predicted: {prediction}")
        print(f"Actual: {true_label}")
        print(f"Correct: {is_correct}\n")

    accuracy = correct / total
    return accuracy

test_cases = [
    {"input": "This product exceeded my expectations!", "label": "Positive"},
    {"input": "I'm utterly disappointed with the service.", "label": "Negative"},
    {"input": "The temperature today is 72 degrees.", "label": "Neutral"}
]

accuracy = evaluate_model(few_shot_sentiment_classification, test_cases)
print(f"Model Accuracy: {accuracy:.2f}")

Input: This product exceeded my expectations!
Predicted: Positive
Actual: Positive
Correct: True

Input: I'm utterly disappointed with the service.
Predicted: Negative
Actual: Negative
Correct: True

Input: The temperature today is 72 degrees.
Predicted: Neutral

The text simply states a fact about the temperature without expressing any emotion or opinion.
Actual: Neutral
Correct: False

Model Accuracy: 0.67
